In [76]:
df1 = df.loc[(df["label"]=='hate') | (df["label"]=='noHate')]

In [16]:
## BOW
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(df1['text'])

In [77]:
from sklearn.model_selection import train_test_split

X = df1.text
y = df1.new_label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [85]:
type(y_train)

pandas.core.series.Series

In [32]:
X_test.shape

(2141, 15576)

In [36]:
from collections import Counter
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])
# df.progress_apply is tqdm method for pandas. It shows progress bar for apply function
# remove the leading and trailing spaces
df['text'] = df.text.progress_apply(lambda x: x.strip())

words = Counter()
for sent in tqdm(df.text.values):
    words.update(w.text.lower() for w in nlp(sent))
   
# sort with most frequently occuring words first
words = sorted(words, key=words.get, reverse=True)
# add <pad> and <unk> token to vocab which will be used later
words = ['_PAD','_UNK'] + words

# create word to index dictionary and reverse
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

100%|██████████████████████████████████████████████████████████████████████████| 10944/10944 [00:06<00:00, 1733.75it/s]


In [86]:
def get_batch(X,y,i,batch_size):
    batches = []
    results = []
    texts = X[i*batch_size:i*batch_size+batch_size]
    categories = y[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        print("text",text)
        layer = np.zeros(total_words,dtype=float)
        for word in text.split(' '):
            layer[word2idx[word.lower()]] += 1

        batches.append(layer)

    for category in categories:
        index_y = -1
        if category == 0:
            index_y = 0
        elif category == 1:
            index_y = 1
        else:
            index_y = 2
        results.append(index_y)


    return np.array(batches),np.array(results)

In [87]:
learning_rate = 0.01
num_epochs = 10
batch_size = 150
display_step = 1

total_words = len(words)
# Network Parameters
hidden_size = 100      # 1st layer and 2nd layer number of features
input_size = total_words # Words in vocab
num_classes = 3  

In [88]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class OurNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(OurNet, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [46]:
input [batch_size, n_labels]
# output [max index for each item in batch, ... ,batch_size-1]

loss = nn.CrossEntropyLoss()
# input = Variable(torch.randn(2, 5), requires_grad=True)
print(">>> batch of size 2 and 5 possible classes")
print(input)
target = Variable(torch.LongTensor(2).random_(5))
print(">>> array of size 'batch_size' with the index of the maxium label for each item")
print(target)
output = loss(input, target)
output.backward()

>>> batch of size 2 and 5 possible classes
tensor([[ 0.0174, -0.7867,  1.0497,  0.0484, -1.1184],
        [ 1.5302,  0.3991,  0.9495,  0.6269,  0.6505]], requires_grad=True)
>>> array of size 'batch_size' with the index of the maxium label for each item
tensor([1, 1])


array([1, 2, 1, ..., 1, 1, 1], dtype=int64)

In [89]:
net = OurNet(input_size, hidden_size, num_classes)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

# Train the Model
for epoch in range(num_epochs):
    total_batch = int((X_train.shape[0])/batch_size)
    print(total_batch)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(X_train,y_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        print("articles",articles)
        print(batch_x, labels)
        print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(X_train)//batch_size, loss.data[0]))

57
text she turns 7 tomorrow .
text egyptians are not arabs , they are egyptians .
text certain individuals like to claim some form of ' moral high-ground ' in our racial fight to preserve the white race. irish republicans hidden jewish past .....


KeyError: 'high-ground'

In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class OurNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.layer_1 = nn.Linear(n_inputs,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [4]:
net = OurNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

NameError: name 'input_size' is not defined

In [7]:
correct = 0
total = 0
total_test_data = len(newsgroups_test.target)
batch_x_test,batch_y_test = get_batch(newsgroups_test,0,total_test_data)
articles = Variable(torch.FloatTensor(batch_x_test))
labels = Variable(torch.LongTensor(batch_y_test))
outputs = net(articles)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()

NameError: name 'newsgroups_test' is not defined

In [6]:
for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    for i in range(total_batch):
        batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.FloatTensor(batch_y))

        # Forward + Backward + Optimize
        optimizer.zero_grad() # zero the gradient buffer
    outputs = net(articles)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.data[0]))

NameError: name 'num_epochs' is not defined

In [3]:
# class TextCNN(nn.Module):
#     def __init__(self, config, vocab_size, word_embeddings):
#         super(TextCNN, self).__init__()
#         self.config = config
        
#         # Embedding Layer
#         self.embeddings = nn.Embedding(vocab_size, self.config.embed_size)
#         self.embeddings.weight = nn.Parameter(word_embeddings, requires_grad=False)
        
#         # This stackoverflow thread clarifies how conv1d works
#         # https://stackoverflow.com/questions/46503816/keras-conv1d-layer-parameters-filters-and-kernel-size/46504997
#         self.conv1 = nn.Sequential(
#             nn.Conv1d(in_channels=self.config.embed_size, out_channels=self.config.num_channels, kernel_size=self.config.kernel_size[0]),
#             nn.ReLU(),
#             nn.MaxPool1d(self.config.max_sen_len - self.config.kernel_size[0]+1)
#         )
#         self.conv2 = nn.Sequential(
#             nn.Conv1d(in_channels=self.config.embed_size, out_channels=self.config.num_channels, kernel_size=self.config.kernel_size[1]),
#             nn.ReLU(),
#             nn.MaxPool1d(self.config.max_sen_len - self.config.kernel_size[1]+1)
#         )
#         self.conv3 = nn.Sequential(
#             nn.Conv1d(in_channels=self.config.embed_size, out_channels=self.config.num_channels, kernel_size=self.config.kernel_size[2]),
#             nn.ReLU(),
#             nn.MaxPool1d(self.config.max_sen_len - self.config.kernel_size[2]+1)
#         )
        
#         self.dropout = nn.Dropout(self.config.dropout_keep)
        
#         # Fully-Connected Layer
#         self.fc = nn.Linear(self.config.num_channels*len(self.config.kernel_size), self.config.output_size)
        
#         # Softmax non-linearity
#         self.softmax = nn.Softmax()
        
#     def forward(self, x):
#         # x.shape = (max_sen_len, batch_size)
#         embedded_sent = self.embeddings(x).permute(1,2,0)
#         # embedded_sent.shape = (batch_size=64,embed_size=300,max_sen_len=20)
        
#         conv_out1 = self.conv1(embedded_sent).squeeze(2) #shape=(64, num_channels, 1) (squeeze 1)
#         conv_out2 = self.conv2(embedded_sent).squeeze(2)
#         conv_out3 = self.conv3(embedded_sent).squeeze(2)
        
#         all_out = torch.cat((conv_out1, conv_out2, conv_out3), 1)
#         final_feature_map = self.dropout(all_out)
#         final_out = self.fc(final_feature_map)
#         return self.softmax(final_out)
    
#     def add_optimizer(self, optimizer):
#         self.optimizer = optimizer
        
#     def add_loss_op(self, loss_op):
#         self.loss_op = loss_op
    
#     def reduce_lr(self):
#         print("Reducing LR")
#         for g in self.optimizer.param_groups:
#             g['lr'] = g['lr'] / 2
                
#     def run_epoch(self, train_iterator, val_iterator, epoch):
#         train_losses = []
#         val_accuracies = []
#         losses = []
        
#         # Reduce learning rate as number of epochs increase
#         if (epoch == int(self.config.max_epochs/3)) or (epoch == int(2*self.config.max_epochs/3)):
#             self.reduce_lr()
            
#         for i, batch in enumerate(train_iterator):
#             self.optimizer.zero_grad()
#             if torch.cuda.is_available():
#                 x = batch.text.cuda()
#                 y = (batch.label - 1).type(torch.cuda.LongTensor)
#             else:
#                 x = batch.text
#                 y = (batch.label - 1).type(torch.LongTensor)
#             y_pred = self.__call__(x)
#             loss = self.loss_op(y_pred, y)
#             loss.backward()
#             losses.append(loss.data.cpu().numpy())
#             self.optimizer.step()
    
#             if i % 100 == 0:
#                 print("Iter: {}".format(i+1))
#                 avg_train_loss = np.mean(losses)
#                 train_losses.append(avg_train_loss)
#                 print("\tAverage training loss: {:.5f}".format(avg_train_loss))
#                 losses = []
                
#                 # Evalute Accuracy on validation set
#                 val_accuracy = evaluate_model(self, val_iterator)
#                 print("\tVal Accuracy: {:.4f}".format(val_accuracy))
#                 self.train()
                
#         return train_losses, val_accuracies

In [61]:
import torch
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.datasets import fetch_20newsgroups


categories = ["comp.graphics","sci.space","rec.sport.baseball"]
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

print('total texts in train:',len(newsgroups_train.data))
print('total texts in test:',len(newsgroups_test.data))

total texts in train: 1774
total texts in test: 1180


In [82]:
type(newsgroups_train.data[0]), newsgroups_train.target[0]

(str, 0)

In [62]:
vocab = Counter()

for text in newsgroups_train.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1

for text in newsgroups_test.data:
    for word in text.split(' '):
        vocab[word.lower()]+=1

total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i

    return word2index

word2index = get_word_2_index(vocab)

In [90]:
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.data[i*batch_size:i*batch_size+batch_size]
    categories = df.target[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        layer = np.zeros(total_words,dtype=float)
        for word in text.split(' '):
            layer[word2index[word.lower()]] += 1

        batches.append(layer)

    for category in categories:
        index_y = -1
        if category == 0:
            index_y = 0
        elif category == 1:
            index_y = 1
#         else:
#             index_y = 2
        results.append(index_y)


    return np.array(batches),np.array(results)

In [91]:
# Parameters
learning_rate = 0.01
num_epochs = 10
batch_size = 150
display_step = 1

# Network Parameters
hidden_size = 100      # 1st layer and 2nd layer number of features
input_size = total_words # Words in vocab
num_classes = 2         # Categories: graphics, sci.space and baseball

In [66]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class OurNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(OurNet, self).__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out

In [92]:
net = OurNet(input_size, hidden_size, num_classes)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)  

# Train the Model
for epoch in range(num_epochs):
    total_batch = int(len(newsgroups_train.data)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(newsgroups_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        #print("articles",articles)
        #print(batch_x, labels)
        #print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(newsgroups_train.data)//batch_size, loss.data[0]))

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at c:\new-builder_3\win-wheel\pytorch\aten\src\thnn\generic/ClassNLLCriterion.c:93

In [70]:
correct = 0
total = 0
total_test_data = len(newsgroups_test.target)
batch_x_test,batch_y_test = get_batch(newsgroups_test,0,total_test_data)
articles = Variable(torch.FloatTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)
outputs = net(articles)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()

print('Accuracy of the network on the 1180 test articles: %d %%' % (100 * correct / total))

Accuracy of the network on the 1180 test articles: 91 %
